In [1]:
import numpy as np
from math import floor, ceil
from random import randint, choice
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from tqdm import tqdm
from tqdm.autonotebook import tqdm
import time as tm
import multiprocessing as mp   
from LAB_UW_functions import *

/tmp/ipykernel_5795/1139959824.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
###### INPUT #######

machine_name = "Brava_2"
experiment_name = "s0108"
data_type = "data_tsv_files"

### Some plot features ###
image_types = ("waveforms","amplitude_maps","amplitude_spectrum_maps", "example_of_filtered_spectrum","example_of_filtered_waveform")
formato = '.png'
remove_initial_samples = 200    # number of samples to be removed at the beginning, to get rid of the noise burst.
highlight_start = 0
highlight_end = 0
xlim_plot= 50
ticks_steps_waveforms = 2 #[microseconds] plot ticks 
step_wf_to_plot = 100  # get one waveforms each step_wf_to_plot 
amp_scale = 300  #color map for amplitude scale
freq_cut = 5   # [Hz] lowpass frequency threshold 


infile_path_list = make_infile_path_list(machine_name,experiment_name,  data_type = data_type)
outdir_path_images = make_images_folders(machine_name, experiment_name, image_types)       

for infile_path in sorted(infile_path_list):
    print('PROCESSING UW DATA IN %s: ' %infile_path)

    data,metadata = make_UW_data(infile_path)
    data,metadata = remove_starting_noise(data, metadata, remove_initial_samples)

    #CHOOSE THE FILE PATH OF THE OUTPUT. Add here a line for each type of images you want and call the relative plot function with the choosen path
    outfile_name = os.path.basename(infile_path).split('.')[0]    # remove extension

    outfile_path_waveforms = os.path.join(outdir_path_images[0], outfile_name)
    outfile_path_amp_map = os.path.join(outdir_path_images[1], outfile_name)
    outfile_path_spectrum_map = os.path.join(outdir_path_images[2], outfile_name)
    outfile_path_example_filtered_spectrum = os.path.join(outdir_path_images[3], outfile_name)
    outfile_path_example_original_vs_filtered = os.path.join(outdir_path_images[4], outfile_name)


    ## PLOT DATA ###
    start_time = tm.time()
    uw_all_plot(data, metadata, step_wf_to_plot, highlight_start, highlight_end, 
                xlim_plot, ticks_steps_waveforms, outfile_path = outfile_path_waveforms, formato = formato)
    amplitude_map(data,metadata, amp_scale, outfile_path=outfile_path_amp_map, formato = formato)

    # FILTER WITH A LOWPASS AND PLOT THE AMPLITUDE MAP OF THE SPECTRUM
    filtered_data, noise_recostructed = signal2noise_separation_lowpass(data, metadata, freq_cut=freq_cut, outfile_path= outfile_path_spectrum_map, formato=formato )

    # PLOT THE FILTERED DATA IN THE SAME FOLDERS OF THE ORIGINAL DATA
    uw_all_plot(filtered_data, metadata, step_wf_to_plot, highlight_start, highlight_end, 
                xlim_plot, ticks_steps_waveforms, outfile_path = outfile_path_waveforms + "_filtered", formato = formato)
    amplitude_map(filtered_data,metadata, amp_scale,outfile_path = outfile_path_amp_map + "_filtered", formato = formato)

    wave_chosed = randint(0,metadata['number_of_waveforms'])
    single_waveform = data[wave_chosed,:]
    filtered_single_waveform, filtered_noise_recostructed = signal2noise_separation_lowpass(single_waveform, metadata, freq_cut=freq_cut, outfile_path=outfile_path_example_filtered_spectrum)
    signal_vs_filtered_signal_plot(single_waveform, filtered_single_waveform, metadata, freq_cut=freq_cut, outfile_path = outfile_path_example_original_vs_filtered, )


    print("--- %s seconds for processing %s---" % (tm.time() - start_time, outfile_name))

        

PROCESSING UW DATA IN /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/data_tsv_files/001_run_in_10MPa.bscan.tsv: 
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/images/waveforms/001_run_in_10MPa0
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/images/waveforms/001_run_in_10MPa1
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/images/waveforms/001_run_in_10MPa2
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/images/waveforms/001_run_in_10MPa3
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/images/waveforms/001_run_in_10MPa4
Plot saved to /home/michele/Desktop/Dottorato/active_source_implementation/experiments_Brava_2/s0108/images/waveforms/001_run_in_10MPa5
Plot saved to /home/michele/Des

: 